In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f" OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print(" OpenAI API Key not set")

openai = OpenAI()
MODEL = "gpt-4o-mini"

market_data = {
    "apple": {"price": "$189", "sentiment": "Bullish"},
    "microsoft": {"price": "$312", "sentiment": "Bearish"},
    "google": {"price": "$141", "sentiment": "Bullish"},
    "tesla": {"price": "$265", "sentiment": "Bearish"},
}

def get_stock_sentiment(company_name):
    print(f"Tool get_stock_sentiment called for {company_name}")
    company = company_name.lower()
    return market_data.get(company, {"price": "Unknown", "sentiment": "Unknown"})

stock_function = {
    "name": "get_stock_sentiment",
    "description": "Get the current stock price and sentiment (bullish or bearish) for a given company.",
    "parameters": {
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": "The company the user is asking about (e.g., Apple, Microsoft, Tesla).",
            },
        },
        "required": ["company_name"],
        "additionalProperties": False,
    },
}

tools = [{"type": "function", "function": stock_function}]

system_message = """
You are a Stock Market Assistant. 
When asked about a company, find the current stock sentiment (bullish or bearish) 
and the market price using the available tool. 
Always explain in simple terms.
"""

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, company = handle_tool_call(message)
        messages.append(message)   # assistant’s tool request
        messages.append(response)  # tool’s result

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    company = arguments.get("company_name")
    stock_info = get_stock_sentiment(company)
    response = {
        "role": "tool",
        "content": json.dumps({"company_name": company, "price": stock_info["price"], "sentiment": stock_info["sentiment"]}),
        "tool_call_id": tool_call.id,
    }
    return response, company

demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="📈 Stock Market Sentiment Assistant",
    description="Ask about a company (e.g., Apple, Tesla) and I’ll tell you if the market is bullish or bearish today."
)

demo.launch()

✅ OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool get_stock_sentiment called for Apple


In [3]:
!pip install yfinance


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 3.7 MB/s  0:00:00 3.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.1 MB/s  0:00:00m 4.2 MB/s eta 0:00:01
  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=c920a53806b866611e18f672a6da83bc961f9faf201132a338141202a40e3a4

In [15]:
# Project 4: Stock Market Sentiment Assistant (Live Data)
# =========================================================

# --- Imports ---
import os
import json
import yfinance as yf
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# --- API Setup ---
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f" OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print(" OpenAI API Key not set")

openai = OpenAI()
MODEL = "gpt-4o-mini"

# Step 1: Write a function
# =========================================================
def get_stock_sentiment(company_name_or_ticker):
    stock = yf.Ticker(company_name_or_ticker.upper())
    history = stock.history(period="2d")

    latest = history.iloc[-1]
    prev = history.iloc[-2]

    sentiment = "Bullish 📈" if latest["Close"] > prev["Close"] else "Bearish 📉"

    return {
        "price": f"${latest['Close']:.2f}",
        "sentiment": sentiment,
        "open": f"${latest['Open']:.2f}",
        "high": f"${latest['High']:.2f}",
        "low": f"${latest['Low']:.2f}",
        "volume": int(latest["Volume"])
    }


# Step 2: Describe the function
# =========================================================
stock_function = {
    "name": "get_stock_sentiment",
    "description": "Always call this tool to get the live stock price and sentiment (bullish or bearish) for any ticker symbol the user asks about.",
    "parameters": {
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": "The ticker symbol of the company (e.g., NVDA for Nvidia, AAPL for Apple).",
            },
        },
        "required": ["company_name"],
        "additionalProperties": False,
    },
}

# Step 3: Add to tools list
# =========================================================
tools = [{"type": "function", "function": stock_function}]

# --- System Message ---
system_message = """
You are a Stock Market Assistant. 
When reporting results, explain in detail — include the stock’s price, sentiment, 
and additional insights (like what bullish/bearish means, possible reasons, 
and beginner-friendly interpretation).
"""



# Step 4: Chat function (LLM may call tool)
# =========================================================
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # If model requests a tool
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, company = handle_tool_call(message)
        messages.append(message)   # assistant tool request
        messages.append(response)  # tool result

        # Send back updated conversation
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


# Step 5: Handle tool call
# =========================================================
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    company = arguments.get("company_name")
    stock_info = get_stock_sentiment(company)
    response = {
        "role": "tool",
        "content": json.dumps({"company_name": company, "price": stock_info["price"], "sentiment": stock_info["sentiment"]}),
        "tool_call_id": tool_call.id,
    }
    return response, company

# Launch Gradio App
# =========================================================
demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="📈 Stock Market Sentiment Assistant",
    description="Ask about Apple, Microsoft, Tesla, or Google to get live price and sentiment."
)

demo.launch(share= True)



 OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://51d79a0b4cdac98c49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
